In [2]:
import tensorflow as tf
import tensorflow.keras as keras
tf.__version__

'1.14.0'

In [6]:
tf.contrib.integrate.odeint?

In [ ]:
class OdeBlock(keras.layers.Layer):
    def __init__(self, nlayers, nunits, output_shape):
        self._nlayers = nlayers
        self._nunits = nunits
        self._output_shape = output_shape
    
    def build(self, input_shape):
        layer = keras.Layers.Input(shape=input_shape)
        self.input = layer
        for i in range(self._nlayers):
            layer = keras.layers.Dense(self._nunits)(layer)
        self.output = keras.layers.Dense(self._output_shape)(layer)
        super(OdeBlock, self).build(input_shape)
    
    def call(self, x):
        t = tf.Constant([0, 1], dtype="float32")
        return tf.contrib.integrate.odeint(self.function, x, t, rtol=1e-3, atol=1e-3)[1]
    
    def _function(self, inputs):
        
            

In [7]:
keras.Layers.Dense?

Object `keras.Layers.Dense` not found.


In [11]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Layer

def set_gpu_config(device = "0",fraction=0.25):
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = fraction
    config.gpu_options.visible_device_list = device
    K.set_session(tf.Session(config=config))


class ODEBlock(Layer):

    def __init__(self, filters, kernel_size, **kwargs):
        self.filters = filters
        self.kernel_size = kernel_size
        super(ODEBlock, self).__init__(**kwargs)

    def build(self, input_shape):
        self.conv2d_w1 = self.add_weight("conv2d_w1", self.kernel_size + (self.filters + 1, self.filters), initializer='glorot_uniform')
        self.conv2d_w2 = self.add_weight("conv2d_w2", self.kernel_size + (self.filters + 1, self.filters), initializer='glorot_uniform')
        self.conv2d_b1 = self.add_weight("conv2d_b1", (self.filters,), initializer='zero')
        self.conv2d_b2 = self.add_weight("conv2d_b2", (self.filters,), initializer='zero')
        super(ODEBlock, self).build(input_shape)

    def call(self, x):
        t = K.constant([0, 1], dtype="float32")
        return tf.contrib.integrate.odeint(self.ode_func, x, t, rtol=1e-3, atol=1e-3)[1]

    def compute_output_shape(self, input_shape):
        return input_shape

    def ode_func(self, x, t):
        y = self.concat_t(x, t)
        y = K.conv2d(y, self.conv2d_w1, padding="same")
        y = K.bias_add(y, self.conv2d_b1)
        y = K.relu(y)

        y = self.concat_t(y, t)
        y = K.conv2d(y, self.conv2d_w2, padding="same")
        y = K.bias_add(y, self.conv2d_b2)
        y = K.relu(y)

        return y

    def concat_t(self, x, t):
        new_shape = tf.concat(
            [
                tf.shape(x)[:-1],
                tf.constant([1],dtype="int32",shape=(1,))
            ], axis=0)

        t = tf.ones(shape=new_shape) * tf.reshape(t, (1, 1, 1, 1))
        return tf.concat([x, t], axis=-1)


def build_model(input_shape, num_classes):
    x = Input(input_shape)
    y = Conv2D(32, (3, 3), activation='relu')(x)
    y = MaxPooling2D((2,2))(y)
    y = Conv2D(64, (3, 3), activation='relu')(y)
    y = MaxPooling2D((2,2))(y)
    y = ODEBlock(64, (3, 3))(y)
    y = Flatten()(y)
    y = Dense(num_classes, activation='softmax')(y)
    return Model(x,y)


set_gpu_config("0",0.25)

batch_size = 128
num_classes = 10
epochs = 10
image_shape = (28, 28, 1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

x_train = x_train.reshape((-1,) + image_shape)
x_test = x_test.reshape((-1,) + image_shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)


model = build_model(image_shape, num_classes)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0211 19:01:42.367102  7452 deprecation.py:506] From c:\program files\python37\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0211 19:01:43.020567  7452 deprecation.py:323] From c:\program files\python37\lib\site-packages\tensorflow\contrib\integrate\python\ops\odes.py:233: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Train on 60000 samples, validate on 10000 samples


W0211 19:01:43.444869  7452 deprecation.py:323] From c:\program files\python37\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
60000/60000 [==============================] - 98s 2ms/sample - loss: 0.2079 - acc: 0.9380 - val_loss: 0.0489 - val_acc: 0.9834
Epoch 2/10
60000/60000 [==============================] - 96s 2ms/sample - loss: 0.0502 - acc: 0.9844 - val_loss: 0.0307 - val_acc: 0.9905
Epoch 3/10
60000/60000 [==============================] - 98s 2ms/sample - loss: 0.0342 - acc: 0.9893 - val_loss: 0.0302 - val_acc: 0.9913
Epoch 4/10
60000/60000 [==============================] - 97s 2ms/sample - loss: 0.0282 - acc: 0.9911 - val_loss: 0.0228 - val_acc: 0.9926
Epoch 5/10
60000/60000 [==============================] - 96s 2ms/sample - loss: 0.0219 - acc: 0.9930 - val_loss: 0.0296 - val_acc: 0.9914
Epoch 6/10
60000/60000 [==============================] - 94s 2ms/sample - loss: 0.0192 - acc: 0.9939 - val_loss: 0.0228 - val_acc: 0.9924
Epoch 7/10
60000/60000 [==============================] - 93s 2ms/sample - loss: 0.0168 - acc: 0.9945 - val_loss: 0.0252 - val_acc: 0.9933
Epoch 8/10
60000/60000 [===